The following is a d3 graph-like tree example. To run the program, you must execute each cell one at a time. If you get the error "d3 not loaded", rerun the first cell (%load_ext py_d3)

In [1]:
%load_ext py_d3

In [2]:
%%d3 4.2.2
<style>

        .node circle {
          fill: #fff;
          stroke: steelblue;
          stroke-width: 3px;
        }

        .node text {
          font: 12px sans-serif;
        }

        .link {
          fill: none;
          stroke: #ccc;
          stroke-width: 2px;
        }

        /* Clunky first https://bl.ocks.org/d3noob/257c360b3650b9f0a52dd8257d7a2d73 */
        div.tooltip {
          position: absolute;
          text-align: center;
          width: 60px;
          height: 28px;
          padding: 2px;
          font: 12px sans-serif;
          background: lightsteelblue;
          border: 0px;
          border-radius: 8px;
          pointer-events: none;
        }
</style>

<g></g>

<script>
function parseNewick(a){for(var e=[],r={},s=a.split(/\s*(;|\(|\)|,|:)\s*/),t=0;t<s.length;t++){var n=s[t];switch(n){case"(":var c={};r.branchset=[c],e.push(r),r=c;break;case",":var c={};e[e.length-1].branchset.push(c),r=c;break;case")":r=e.pop();break;case":":break;default:var h=s[t-1];")"==h||"("==h||","==h?r.name=n:":"==h&&(r.length=parseFloat(n))}}return r}

var margin = {top: 20, right: 90, bottom: 30, left: 90},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

// append the svg object to the body of the page
// appends a 'group' element to 'svg'
// moves the 'group' element to the top left margin
var svg = d3.select("g").append("svg")
    .attr("width", width + margin.right + margin.left)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate("
          + margin.left + "," + margin.top + ")");

var i = 0,
    duration = 750,
    root;
    

 // Create tool tip   
var div = svg.append("div")
        .attr("class", "tooltip")
        .style("opacity", 0);


// declares a tree layout and assigns the size
var treemap = d3.tree().size([height, width]);

var treemat = [1];

d3.queue()
    .defer(d3.text, "kate_output_slack.txt")
    .defer(d3.csv, "breast_cancer_prim_perf_counter_data.csv")
    .await(analyze);
    
function analyze(error, treeformat, perfdata) {
    if (error) throw error;
    
    // Assigns parent, children, height, depth
    root = d3.hierarchy(parseNewick(treeformat), function(d) { return d.branchset; });
    root.x0 = height / 2;
    root.y0 = 0;

    // Collapse after the second level
    //root.children.forEach(collapse); //katy
    
    // Collecting the performance times
    domainTimes = [];
    prim_inst = [];
    perfdata.map(function(d){
        var sumTimes = (+d.time) + (+d.direct_time);
        if (sumTimes > 0) {
            domainTimes.push((+d.time) + (+d.direct_time));
        }
        
        prim_inst.push(d.primitive_instance);
    });
    colorTimeScale = d3.scaleLog() //went with log scale
                .domain(d3.extent(domainTimes))
                .range(["#deebf7", "#9ecae1", "#3182bd"]);
    
    update(root, perfdata);
    //count = 1; //count the root
    //countNodes(root);
    
}



// Collapse the node and all it's children
function collapse(d) {
  if(d.children) {
    d._children = d.children;
    d._children.forEach(collapse);
    d.children = null;
  }
}

// Count the number of nodes at each level
function countNodes(node) {
    if(node.children){
        for(var i=0; i<node.children.length; i++) {
            if (node.children[i].children){
                count++;
                countNodes(node.children[i]);
            }
            else {
                count++;
            }
        }
    }
}


function update(source, perfdata) {

  // Assigns the x and y position for the nodes
  var tree = treemap(root);

  // Compute the new tree layout.
  var nodes = tree.descendants(),
      links = tree.descendants().slice(1);

  // Compute the maximum depth of the tree katy
  var maxDepth=0; 
  nodes.forEach(function(d){
      if (d.depth > maxDepth){ 
        maxDepth = d.depth;
      } 
  });

  // Normalize for fixed-depth. katy
  nodes.forEach(function(d){ 
        depthY = width / maxDepth;
      d.y = d.depth * depthY;
  }); //was 180. Made dependent on # of levels and width
    


  // ****************** Nodes section ***************************

  // Update the nodes...
  var node = svg.selectAll('g.node')
      .data(nodes, function(d) {return d.id || (d.id = ++i); });

  // Enter any new modes at the parent's previous position.
  var nodeEnter = node.enter().append('g')
      .attr('class', 'node')
      .attr("transform", function(d) {
        return "translate(" + source.y0 + "," + source.x0 + ")";
    })
    .on('click', click);

  // Add Circle for the nodes
  nodeEnter.append('circle')
      .attr('class', 'node')
      .attr('r', 1e-6)
      .style("fill", function(d) {
          return d._children ? "lightsteelblue" : "#fff";
      });

  // Add labels for the nodes
  nodeEnter.append('text')
      .attr("dy", ".35em")
      .attr("x", function(d) {
          return d.children || d._children ? -13 : 13;
      })
      .attr("text-anchor", function(d) {
          return d.children || d._children ? "end" : "start";
      })
      .text(function(d) {           
          if (!d.children) {
              return d.data.name;
          }  
      });

  // UPDATE
  var nodeUpdate = nodeEnter.merge(node);

  // Transition to the proper position for the node
  nodeUpdate.transition()
    .duration(duration)
    .attr("transform", function(d) { 
        return "translate(" + d.y + "," + d.x + ")";
     });

  // Update the node attributes and style
  nodeUpdate.select('circle.node')
    .attr('r', 10)
    .style("fill", function(d) {
        return d._children ? "lightsteelblue" : "#fff";
    })
    .attr('cursor', 'pointer');


  // Remove any exiting nodes
  var nodeExit = node.exit().transition()
      .duration(duration)
      .attr("transform", function(d) {
          return "translate(" + source.y + "," + source.x + ")";
      })
      .remove();

  // On exit reduce the node circles size to 0
  nodeExit.select('circle')
    .attr('r', 1e-6);

  // On exit reduce the opacity of text labels
  nodeExit.select('text')
    .style('fill-opacity', 1e-6);

  // ****************** links section ***************************

  // Update the links...
  var link = svg.selectAll('path.link')
      .data(links, function(d) { return d.id; });

  // Enter any new links at the parent's previous position.
  var linkEnter = link.enter().insert('path', "g")
      .attr("class", "link")
      .attr('d', function(d){
        var o = {x: source.x0, y: source.y0};
        return diagonal(o, o);
      });

  // UPDATE
  var linkUpdate = linkEnter.merge(link);

  // Transition back to the parent element position
  linkUpdate.transition()
      .duration(duration)
      .attr('d', function(d){ return diagonal(d, d.parent) });

  // Remove any exiting links
  var linkExit = link.exit().transition()
      .duration(duration)
      .attr('d', function(d) {
        var o = {x: source.x, y: source.y}
        return diagonal(o, o)
      })
      .remove();

  // Store the old positions for transition.
  nodes.forEach(function(d){
    d.x0 = d.x;
    d.y0 = d.y;
  });

  // Creates a curved (diagonal) path from parent to the child nodes
  function diagonal(s, d) {

    path = `M ${s.y} ${s.x}
            C ${(s.y + d.y) / 2} ${s.x},
              ${(s.y + d.y) / 2} ${d.x},
              ${d.y} ${d.x}`;

    return path
  }

  // Toggle children on click.
  function click(d) {
    var t_cell = IPython.notebook.insert_cell_below()
    t_cell.set_text('count = ' + d.count);//<!--\\n' + t_cell.get_text() + '\\n--> \\n{}');
    var t_index = IPython.notebook.get_cells().indexOf(t_cell);
    //IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();

      
    if (d.children) {
        d._children = d.children;
        d.children = null;
      } else {
        d.children = d._children;
        d._children = null;
      }
    update(d, perfdata);
  }
} 


var t_cell = IPython.notebook.insert_cell_below()
    t_cell.set_text('count = 22 ');//<!--\\n' + t_cell.get_text() + '\\n--> \\n{}');
    var t_index = IPython.notebook.get_cells().indexOf(t_cell);
    //IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();

</script>

In [ ]:
count = 22 

In [ ]:
count = 22 

In [ ]:
count = undefined

In [ ]:
count = 22 

In [ ]:
count = 22 